In [1]:
from deepeval.synthesizer import Synthesizer 
from deepeval.synthesizer.config import ContextConstructionConfig,StylingConfig

import os
from dotenv import load_dotenv

load_dotenv()

# Initialize the synthesizer with custom LLM and embedding models
def synthesize():
    """
    This function initializes the synthesizer 
    and generates golden outputs from documents.
    """
    styling_config=StylingConfig(
            input_format="Questions about homewoner insurance policy. What cover/not cover depending on home damage situation",
            expected_output_format="Detail answer on what covered/not covered for that accident based on policy",
            task="RAG chatbot for homewner insurance policy documents",
            scenario="Customer asking queries about their policy information"

        )
    
    synthesizer = Synthesizer(cost_tracking=True,styling_config=styling_config)

    goldens = synthesizer.generate_goldens_from_docs(
        document_paths=["../../evaluation/data/docs/sample_policy_doc_AU1234.pdf"],
        include_expected_output=True,
        max_goldens_per_context=2,  # Limit to one golden per context
        context_construction_config=ContextConstructionConfig(
            max_contexts_per_document=1,  # Limit to one context per document 
        )     
    )

    return synthesizer, goldens

synthesizer,goldens = synthesize()


c:\Users\sunny\Desktop\temp_env\env2\venv\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[Confident AI Synthesizer Log] SUCCESS: Successfully deleted: .vector_db

[Confident AI Synthesizer Log] SUCCESS: Successfully deleted: .vector_db

[Confident AI Synthesizer Log] SUCCESS: Context Construction: Utilizing 3 out of 58 chunks.

In [8]:
df = synthesizer.to_pandas()


In [11]:
df.to_excel('dataset.xlsx')

In [2]:
goldens

[Golden(input='What steps should I take after filing a homeowner insurance claim to ensure everything is processed smoothly?', actual_output=None, expected_output='Detail answer on what covered/not covered for that accident based on policy, and make sure it is concise and straight to the point, using supporting information in context.\n\nAfter filing a homeowner insurance claim, you should provide original purchase receipts or relevant documentation to prove your claim, obtain estimates for repairs, and assist with any enquiries regarding your claim. Do not admit responsibility, make agreements if held liable by others, abandon your property, or dispose of damaged items as we may need to see them. We may choose to repair, reinstate, replace, or pay you for loss/damage but not beyond the sum insured. Excess will be deducted from the claim. Report theft or damages due to theft to Police and obtain a reference number. If required, allow us to access your property for assessment and salvag

In [3]:
from deepeval.test_case import LLMTestCase
from deepeval.dataset import Golden
from deepeval import evaluate
from deepeval.metrics import (
    AnswerRelevancyMetric,
    FaithfulnessMetric,
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric
)

In [4]:
import sys
sys.path.append(os.path.abspath("../../"))

In [5]:
from evaluation.rag import RagPipeline
rag_obj = RagPipeline()
test_cases = []
for golden in goldens:
    res,text_chunks = rag_obj.execute_rag(golden.input)
    test_case = LLMTestCase(
        input=golden.input,
        actual_output=res,
        retrieval_context=text_chunks,
        expected_output=golden.expected_output,
    )
    test_cases.append(test_case)

c:\Users\sunny\Desktop\temp_env\env2\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
test_cases

[LLMTestCase(input='What steps should I take after filing a homeowner insurance claim to ensure everything is processed smoothly?', actual_output="After filing a homeowner insurance claim, there are several steps you can take to ensure the process goes smoothly:\n\n1. **Document Everything**: Keep a detailed record of all correspondence with your insurance company, including emails, letters, and notes from phone conversations. Document any expenses incurred as a result of the incident.\n\n2. **Meet Deadlines**: Adhere to any deadlines provided by your insurance company for submitting documents or additional information. This could include estimates for repair or restoration work.\n\n3. **Provide Accurate Information**: Make sure all information you provide is accurate and complete to avoid any delays in processing your claim.\n\n4. **Take Notes During Inspections**: If an insurance adjuster comes to inspect the damage, take notes during their visit. Ask for their contact information to

In [8]:
import json

In [9]:
with open("../../data/goldens/synthetic_dataset.json") as file:
    goldens = json.load(file)

In [12]:
golden = goldens[0]

In [16]:
type(golden)

dict

In [13]:
from evaluation.rag import RagPipeline

In [18]:
rag_obj = RagPipeline()
test_cases = []
res,text_chunks = rag_obj.execute_rag(golden["input"])
test_case = LLMTestCase(
        input=golden["input"],
        actual_output=res,
        retrieval_context=text_chunks,
        expected_output=golden["expected_output"],
    )
test_cases.append(test_case)

In [19]:
test_cases

[LLMTestCase(input='What steps should I take if my home assistance cover is unavailable during an emergency? Does my homeowner insurance policy provide any alternative coverage for such situations?', actual_output="If your home assistance cover is unavailable during an emergency, you can still seek help through your homeowner insurance policy. Here's what you can do:\n\n1. **Contact the Domestic Emergency Helpline**: If you have not taken home assistance cover, you should call the domestic emergency helpline at 01737 815927. AXA Assistance (UK) Limited will connect you with the nearest suitable tradesperson.\n\n2. **Liability and Payment**: Since this is a referral service, any agreement between you and the tradesperson is independent of AXA Assistance. You will be responsible for agreeing on the scope, quality, and cost of the work with the tradesperson.\n\n3. **Submit a Claim**: If the incident is covered by your homeowner insurance policy, you may submit a claim in the usual way for

In [20]:
results = evaluate(
            test_cases=test_cases,
            metrics=[
                AnswerRelevancyMetric()
                
            ]
        )

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

c:\Users\sunny\Desktop\temp_env\env2\venv\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Metrics Summary

  - ✅ Answer Relevancy (score: 0.6666666666666666, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 0.67 because while the response includes some relevant information about emergency contacts and services, it fails to directly address the specific question about alternative coverage under the homeowner insurance policy. The inclusion of unrelated details about tradespeople and AXA Assistance services detracts from the focus on insurance coverage, preventing a higher score., error: None)

For test case:

  - input: What steps should I take if my home assistance cover is unavailable during an emergency? Does my homeowner insurance policy provide any alternative coverage for such situations?
  - actual output: If your home assistance cover is unavailable during an emergency, you can still seek help through your homeowner insurance policy. Here's what you can do:

1. **Contact the Domestic Emergency Helpline**: If you have not taken home assi

✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

In [45]:
res3,res2=results

In [46]:
res3

('test_results',
 [TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.5, success=True, score=0.6666666666666666, reason='The score is 0.67 because while the response includes some relevant information about emergency contacts and services, it fails to directly address the specific question about alternative coverage under the homeowner insurance policy. The inclusion of unrelated details about tradespeople and AXA Assistance services detracts from the focus on insurance coverage, preventing a higher score.', strict_mode=False, evaluation_model='gpt-4o', error=None, evaluation_cost=0.009142500000000001, verbose_logs='Statements:\n[\n    "If home assistance cover is unavailable during an emergency, seek help through your homeowner insurance policy.",\n    "Call the domestic emergency helpline at 01737 815927 if you have not taken home assistance cover.",\n    "AXA Assistance (UK) Limited will connect you with the nearest suitable tr

In [47]:
res2

('confident_link', None)

In [33]:
res,_ = results

In [40]:
_,res1=res

In [73]:
res1

('test_results',
 [TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.5, success=True, score=0.6666666666666666, reason='The score is 0.67 because while the response includes some relevant information about emergency contacts and services, it fails to directly address the specific question about alternative coverage under the homeowner insurance policy. The inclusion of unrelated details about tradespeople and AXA Assistance services detracts from the focus on insurance coverage, preventing a higher score.', strict_mode=False, evaluation_model='gpt-4o', error=None, evaluation_cost=0.009142500000000001, verbose_logs='Statements:\n[\n    "If home assistance cover is unavailable during an emergency, seek help through your homeowner insurance policy.",\n    "Call the domestic emergency helpline at 01737 815927 if you have not taken home assistance cover.",\n    "AXA Assistance (UK) Limited will connect you with the nearest suitable tr

In [ ]:
rows = []
for case in res1:
    print(case.name)
    print(case.input)
    print(case.expected_output)
    print(case.actual_output)
    print(case.success)
   # print(case.passed)
    for met in case.metrics_data:
        print(met.name)
        print(met.score)
        print(met.cost)
        print(met.reason)
    


test_case_0
What steps should I take if my home assistance cover is unavailable during an emergency? Does my homeowner insurance policy provide any alternative coverage for such situations?
Detail answer on what covered/not covered for that accident based on policy, and make sure it is concise and straight to the point, using supporting information in context.

If your home assistance cover is unavailable during an emergency, you can phone the domestic emergency helpline on 01737 815927. AXA Assistance (UK) Limited will provide a referral to the nearest suitable tradesperson. You will be responsible for paying the tradesperson’s fees, but if the incident is covered by your homeowner insurance policy, you may submit a claim in the usual way. The homeowner insurance policy does not cover the tradesperson’s fees as it is a referral service; however, reimbursement may be possible if the incident aligns with policy coverage.
If your home assistance cover is unavailable during an emergency, 

In [71]:
from datetime import datetime
import pandas as pd
import json

In [74]:
#Creating file with current timestamp
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
version_dir = f"../../data/evaluation_results/{timestamp}"
os.makedirs(version_dir, exist_ok=True)

res1,res2=results
_,res3=res1
#Extracting test results in list
rows = []
for case in res3:
    for met in case.metrics_data:
        rows.append({
            "TestCase":case.name,
            "Input": case.input,
            "Expected Output": case.expected_output,
            "Actual Output": case.actual_output,
            "Success":case.success,
            "Matrics": met.name,
            "Score": met.score,
            "Reason": met.reason,
            "Cost":met.evaluation_cost
        })

#Creating csv file if not exists
if not os.path.exists(os.path.dirname(f"{version_dir}/results.csv")):
    os.makedirs(os.path.dirname(f"{version_dir}/results.csv"))

#saving data to csv file
print("💾 Saving results...")
pd.DataFrame(rows).to_csv(f"{version_dir}/results.csv", index=False)

#Creating file if not exists
if not os.path.exists(os.path.dirname(f"{version_dir}/results.json")):
    os.makedirs(os.path.dirname(f"{version_dir}/results.json"))
#saving data to json file
with open(f"{version_dir}/results.json", "w") as f:
    json.dump(rows, f, indent=2)

#Creating file if not exists
if not os.path.exists(os.path.dirname(f"{version_dir}/results.md")):
    os.makedirs(os.path.dirname(f"{version_dir}/results.md"))
#saving data to md file
with open(f"{version_dir}/results.md", "w") as f:
    f.write("# RAG Evaluation Report\n")
    f.write(f"**Run Time**: {timestamp}\n")
    f.write(f"**Dataset**: {version_dir}/results.md\n")
    f.write(f"**LLM Model**: gpt \n\n")

    for i, case in enumerate(res3, 1):
        f.write(f"### Test Case {i}\n")
        f.write(f"- **Input**: {case.input}\n")
        f.write(f"- **Expected**: {case.expected_output}\n")
        f.write(f"- **Actual**: {case.actual_output}\n")
        f.write(f"- **Success**: {case.success}\n")
        for met in case.metrics_data:
            f.write(f"#### Metrics Data \n")
            f.write(f"- **Metrics**: {met.name}\n")
            f.write(f"- **Score**: {met.score}\n")
            f.write(f"- *Cost**: {met.evaluation_cost:.4f}$\n\n")
            f.write(f"- **Reason**: {met.reason}\n\n")

print("✅ Evaluation completed.")
print(f"🟢 Passed {sum(c.success for c in res3)}/{len(res3)} test cases")

💾 Saving results...
✅ Evaluation completed.
🟢 Passed 1/1 test cases
